# Instagram Quran Post Automation

In [3]:
# Step 1: Install required packages
# %pip install requests yt-dlp moviepy pillow instagrapi opencv-python
# %pip install python-dotenv

In [4]:
# Step 2: Import required libraries
from dotenv import load_dotenv
import os
import time
import requests
import subprocess
import random
import cv2
import numpy as np
from datetime import datetime
from PIL import Image, ImageDraw, ImageFont
import yt_dlp
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip, AudioFileClip, concatenate_videoclips, TextClip
from instagrapi import Client
from instagrapi.exceptions import LoginRequired

In [5]:
# Step 3: Configure API keys and credentials
load_dotenv()
UNSPLASH_API_KEY = os.getenv("UNSPLASH_API_KEY") # Replace with your Unsplash API key in an .env file
INSTAGRAM_USERNAME = os.getenv("INSTA_USERNAME")  # Replace with your Instagram username in an .env file 
INSTAGRAM_PASSWORD =  os.getenv("INSTA_PASSWORD") # Replace with your Instagram password in an .env file 
YOUTUBE_CHANNEL_URL = 'https://www.youtube.com/@Am9li9'

# Create directories for storing files
os.makedirs('assets', exist_ok=True)
os.makedirs('output', exist_ok=True)

# Track downloaded videos to avoid duplicates
DOWNLOADED_VIDEOS_FILE = 'assets/downloaded_videos.txt'

In [ ]:
# Step 4: Function to download nature image from Unsplash
def download_nature_image():
    print("Downloading nature image...")
    
    # Unsplash API URL for random photo
    url = 'https://api.unsplash.com/photos/random'
    
    # Parameters for the API request - using vertical orientation for Instagram
    params = {
        'query': 'nature landscape',  # Search for beautiful nature images
        'orientation': 'portrait',     # Vertical for Instagram
        'client_id': UNSPLASH_API_KEY
    }
    
    try:
        # Send the GET request to Unsplash API
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            # Parse the JSON response
            image_data = response.json()
            image_url = image_data['urls']['full']
            image_author = image_data['user']['name']
            
            # Download the image
            img_data = requests.get(image_url).content
            image_path = 'assets/nature_image.jpg'
            with open(image_path, 'wb') as handler:
                handler.write(img_data)
            print(f"Image downloaded successfully from {image_author}!")
            return image_path
        else:
            print(f"Failed to download image: {response.status_code} {response.text}")
            # Fallback to a default image if API fails
            return 'assets/default_nature.jpg'
    except Exception as e:
        print(f"Error downloading image: {e}")
        # Fallback to a default image if any error occurs
        return 'assets/default_nature.jpg'

In [ ]:
# Step 5: Function to download Quran video with text

def download_quran_video():
    print("Downloading Quran video...")
    
    # Load already downloaded video IDs to avoid duplicates
    if os.path.exists(DOWNLOADED_VIDEOS_FILE):
        with open(DOWNLOADED_VIDEOS_FILE, 'r') as f:
            downloaded_video_ids = set(f.read().splitlines())
    else:
        downloaded_video_ids = set()

    # yt-dlp options with more flexible format selection
    ydl_opts = {
        'format': 'best',  # Best quality
        'format_sort': ['res', 'ext:mp4:m4a'],  # Prefer mp4
        'noplaylist': True,  # Ignore playlists
        'outtmpl': 'assets/quran_video.mp4',  # Save with this filename
        'quiet': False,  # Set to False for debugging
        'no_warnings': False,  # Set to False for debugging
        "playlistend": 1,  # Limit to the first video in the playlist
    }

    # Create a yt-dlp instance and download
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Extract channel information first
            info_extraction_opts = ydl_opts.copy()
            info_extraction_opts['quiet'] = True
            info_extraction_opts['no_warnings'] = True
            
            with yt_dlp.YoutubeDL(info_extraction_opts) as info_ydl:
                channel_info = info_ydl.extract_info(YOUTUBE_CHANNEL_URL, download=False)
            
            # Filter videos that contain "سورة" (Surah) in the title
            for entry in channel_info.get('entries', []):
                if entry and 'title' in entry and 'سورة' in entry['title'] and entry['id'] not in downloaded_video_ids:
                    print(f"Found new Quran video: {entry['title']}")
                    
                    # Download this video
                    ydl.download([entry['webpage_url']])
                    
                    # Record that we've downloaded this video
                    with open(DOWNLOADED_VIDEOS_FILE, 'a') as f:
                        f.write(entry['id'] + '\n')
                    
                    return 'assets/quran_video.mp4', entry['title']
            
            # If no new videos, use a random one we haven't posted recently
            all_entries = [e for e in channel_info.get('entries', []) if e and 'title' in e]
            if all_entries:
                random_entry = random.choice(all_entries)
                print(f"Using random Quran video: {random_entry['title']}")
                ydl.download([random_entry['webpage_url']])
                return 'assets/quran_video.mp4', random_entry['title']
            else:
                print("No videos found in the channel.")
                return None, None
    except Exception as e:
        print(f"Error downloading video: {e}")
        # For debugging
        import traceback
        traceback.print_exc()
        return None, None

In [ ]:
# Step 6: Extract text frames from Quran video

def extract_text_from_video(video_path):
    print("Processing video to remove black background...")
    
    # Define the output path (same as before except now using .mov to support transparency)
    processed_video_path = 'assets/quran_text_frame.mov'
    
    # Use FFmpeg to remove the black background:
    # - The filter 'colorkey=black:0.3:0.1' removes black (you can adjust thresholds if needed)
    # - 'format=rgba' ensures the output has an alpha channel
    # - The QuickTime Animation codec (qtrle) supports alpha transparency
    import subprocess
    command = [
        'ffmpeg',
        '-i', video_path,
        '-vf', 'colorkey=black:0.3:0.1,format=rgba',
        '-c:v', 'qtrle',
        '-c:a', 'copy',
        processed_video_path
    ]
    
    subprocess.run(command, check=True)
    print(f"✅ Processed video saved with transparent background: {processed_video_path}")
    
    return processed_video_path


In [ ]:
# Step 7: Create final video with image background and Quran audio/text

def create_final_video(image_path, quran_video_path, text_frame_path=None):
    print("Creating final video...")
    
    try:
        output_path = 'output/final_output.mp4'
        
        # Extract audio from the original Quran video
        audio_path = 'assets/quran_audio.mp3'
        video_clip = VideoFileClip(quran_video_path)
        video_clip.audio.write_audiofile(audio_path, codec='mp3')
        
        # Load the background image
        background = ImageClip(image_path)
        
        # Set the duration to match the extracted audio
        audio_clip = AudioFileClip(audio_path)
        duration = audio_clip.duration
        
        # Resize the background image to a vertical 9:16 format (1080x1920 for Instagram)
        background = background.resize(height=1920)
        if background.w > 1080:
            x_center = background.w / 2
            background = background.crop(x1=x_center-540, y1=0, x2=x_center+540, y2=1920)
        background = background.set_duration(duration)
        
        # Apply a slight dim effect to the background for improved text/overlay visibility
        background = background.fl_image(lambda frame: np.array(frame) * 0.7)
        
        clips = [background]
        
        # If a processed overlay (with transparent background) exists, load it as a VideoClip
        if text_frame_path:
            text_clip = VideoFileClip(text_frame_path, has_mask=True)
            text_clip = text_clip.resize(width=1000).set_position('center').set_duration(duration)
            clips.append(text_clip)
        
        # Composite the background and overlay video clips
        final_clip = CompositeVideoClip(clips)
        
        # Set the audio from the original video
        final_clip = final_clip.set_audio(audio_clip)
        
        # Write the final output video to file
        final_clip.write_videofile(
            output_path,
            codec='libx264',
            audio_codec='aac',
            fps=30,
            preset='fast',
            threads=4
        )
        
        # Release resources
        video_clip.close()
        audio_clip.close()
        final_clip.close()
        
        return output_path
    except Exception as e:
        print(f"Error creating final video: {e}")
        return None


In [10]:
# # Step 8: Post to Instagram
def post_to_instagram(video_path, caption):
    print("Posting to Instagram...")
    
    try:
        # Initialize the Instagram client
        cl = Client()
        
        # Try to load session if exists
        session_file = "instagram_session.json"
        if os.path.exists(session_file):
            try:
                cl.load_settings(session_file)
                cl.get_timeline_feed()  # Test if session is valid
                print("Successfully loaded existing session")
            except LoginRequired:
                print("Session expired, logging in again")
                cl.login(INSTAGRAM_USERNAME, INSTAGRAM_PASSWORD)
                cl.dump_settings(session_file)
        else:
            # Login with username and password
            cl.login(INSTAGRAM_USERNAME, INSTAGRAM_PASSWORD)
            cl.dump_settings(session_file)
        
        # Upload the video as a reel
        media = cl.clip_upload(
            video_path,
            caption=caption,
            thumbnail=None,  # Auto-generate thumbnail
            extra_data={
                "custom_accessibility_caption": "Quran Verse",
                "like_and_view_counts_disabled": False,
                "disable_comments": False
            }
        )
        
        print(f"Successfully posted! Media ID: {media.id}")
        return True
    except Exception as e:
        print(f"Error posting to Instagram: {e}")
        return False

In [11]:
# Step 9: Main function to coordinate the process
def create_and_post_quran_content():
    current_date = datetime.now().strftime("%Y-%m-%d")
    print(f"Starting Quran content creation process - {current_date}")
    
    # 1. Download a beautiful nature image
    image_path = download_nature_image()
    if not image_path:
        return False
    
    # 2. Download a Quran video
    quran_video_path, video_title = download_quran_video()
    # quran_video_path, video_title = 'assets/quran_video.mp4', 'title'    ------> used for testing
    
    if not quran_video_path:
        return False
    
    # 3. Extract text frame from the video
    text_frame_path = extract_text_from_video(quran_video_path)
    
    # 4. Create the final video
    final_video_path = create_final_video(image_path, quran_video_path, text_frame_path)
    if not final_video_path:
        return False
    
    # 5. Generate caption with the verse information and hashtags
    # if video_title:
    #     caption = f"{video_title}\n\n#Quran #Islam #QuranVerses #DailyReminder #Faith #Peace"
    # else:
    #     caption = f"⚠️لا تنسوا اخواننا المستضعفين بالدعاء رحمكم الله⚠️\n\n#اكتب_شي_تؤجر_عليه #لاتنسى_ذكر_الله\n\n#الله #اكتب_شي_تؤجر_عليه #الله_أكبر #قران_كريم #لاتنسى_ذكر_الله #تلاوات #اللهم_امين"
    caption = f"⚠️لا تنسوا اخواننا المستضعفين بالدعاء رحمكم الله⚠️\n\n#اكتب_شي_تؤجر_عليه #لاتنسى_ذكر_الله\n\n#الله #اكتب_شي_تؤجر_عليه #الله_أكبر #قران_كريم #لاتنسى_ذكر_الله #تلاوات #اللهم_امين"
        
    
    # 6. Post to Instagram
    success = post_to_instagram(final_video_path, caption)
    
    if success:
        print("Daily Quran post completed successfully!")
    else:
        print("Failed to complete the daily Quran post.")
    
    return success

In [ ]:
# Step 10: Run the entire process
if __name__ == "__main__":
    create_and_post_quran_content()

Starting Quran content creation process - 2025-04-18


In [ ]:
# For scheduling daily posts:
# 
# On Linux/Mac, add this to crontab (crontab -e):
# 0 9 * * * cd /path/to/your/script && python3 -c "from quran_instagram_automation import create_and_post_quran_content; create_and_post_quran_content()"
#
# On Windows, create a batch file with:
# @echo off
# cd /path/to/your/script
# python -c "from quran_instagram_automation import create_and_post_quran_content; create_and_post_quran_content()"
#
# Then use Task Scheduler to run it daily